In [13]:
import tkinter as tk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [14]:
breast_cancer = load_breast_cancer()
df = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)
df['target'] = breast_cancer.target

df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [15]:
X = df.drop(['target'],axis=1)
y = df['target']

In [16]:
X_train,X_test,y_train,y_test =train_test_split(X, y, test_size=0.2, random_state=42)
classifier = SVC(kernel='poly')
classifier.fit(X_train, y_train)
y_predict = classifier.predict(X_test)

In [17]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       1.00      0.86      0.93        43
           1       0.92      1.00      0.96        71

    accuracy                           0.95       114
   macro avg       0.96      0.93      0.94       114
weighted avg       0.95      0.95      0.95       114



In [18]:
preview = pd.DataFrame(data={
    'Predicted results': y_predict,
    'Actual results': y_test
})

preview

,Predicted results,Actual results
204,1,1
70,0,0
131,0,0
431,1,1
540,1,1
...,...,...
486,1,1
75,0,0
249,1,1
238,1,1


In [19]:
import joblib

# Sauvegarder le modèle SVM
joblib.dump(classifier, 'svm_model.pkl')


['svm_model.pkl']

In [30]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import joblib
import random

# Charger le modèle SVM sauvegardé
classifier = joblib.load('svm_model.pkl')

# Charger le jeu de données sur le cancer du sein
data = pd.DataFrame(load_breast_cancer().data, columns=load_breast_cancer().feature_names)
target = load_breast_cancer().target

def classify_cancer():
    # Obtenir les valeurs saisies dans les champs de saisie
    values = []
    for entry in entries:
        value = entry.get().strip()  # Supprimer les espaces blancs avant et après la saisie
        if value == "":
            result_text.delete(1.0, tk.END)  # Effacer le contenu précédent
            result_text.insert(tk.END, "Veuillez remplir tous les champs de saisie.")
            return  # Sortir de la fonction si l'un des champs est vide
        values.append(float(value))
    
    # Faire une prédiction avec le modèle SVM
    prediction = classifier.predict([values])[0]
    
    # Traduire la prédiction en texte
    prediction_text = "Maligne" if prediction == 1 else "Bénigne"
    
    # Afficher la prédiction dans le champ de texte
    result_text.delete(1.0, tk.END)  # Effacer le contenu précédent
    result_text.insert(tk.END, f"Prédiction: {prediction_text}")

# Fonction pour quitter l'application
def quit_app():
    root.destroy()

# Fonction pour remplir les champs de saisie avec des valeurs aléatoires
def fill_random_values():
    for entry in entries:
        random_values = [random.random() for _ in range(len(feature_names))]
        for entry, value in zip(entries, random_values):
            entry.delete(0, tk.END)
            entry.insert(0, str(value))

# Fonction de validation des entrées des champs de saisie
def validate_input(action, index, value_if_allowed,
                   prior_value, text, validation_type, trigger_type, widget_name):
    if action == '1':  # Text insertion
        if text.replace('.', '', 1).isdigit() or text == "":  # Vérifie si le texte est un nombre réel
            return True
        else:
            return False
    else:
        return True

# Fonction appelée lors du changement d'onglet
def tab_changed(event):
    # Récupérer l'index de l'onglet actif
    current_tab = event.widget.select()
    tab_text = event.widget.tab(current_tab, "text")
    if tab_text == "Quitter":
        quit_app()

# Créer la fenêtre principale
root = tk.Tk()
root.title("Prédiction de cancer du sein")

# Créer un notebook pour les onglets
notebook = ttk.Notebook(root)
notebook.pack(fill='both', expand=True)

# Créer un cadre pour le premier onglet
tab1 = ttk.Frame(notebook)
notebook.add(tab1, text='Prédiction')

# Créer un cadre pour le deuxième onglet
tab2 = ttk.Frame(notebook)
notebook.add(tab2, text='Aide')

# Créer un cadre pour le troisième onglet
tab3 = ttk.Frame(notebook)
notebook.add(tab3, text='Quitter')

# Créer les champs de saisie pour les caractéristiques (onglet 1)
entries = []
for i, feature_name in enumerate(feature_names):
    label = ttk.Label(tab1, text=feature_name + ":")
    label.grid(row=i % 15, column=(i // 15) * 2, padx=10, pady=5, sticky="w")
    entry = ttk.Entry(tab1, validate="key", validatecommand=(root.register(validate_input), '%d', '%i', '%P', '%s', '%S', '%v', '%V', '%W'))
    entry.grid(row=i % 15, column=(i // 15) * 2 + 1, padx=10, pady=5)
    entries.append(entry)

# Créer le champ de texte pour afficher le résultat de la prédiction (onglet 1)
result_text = tk.Text(tab1, height=2, width=30)
result_text.grid(row=len(feature_names)+1, columnspan=2, padx=10, pady=5)

# Créer un bouton pour classer la prédiction (onglet 1)
classify_button = ttk.Button(tab1, text="Classify", command=classify_cancer)
classify_button.grid(row=17, column=0, padx=10, pady=5)  # Place the Classify button at the left

fill_random_button = ttk.Button(tab1, text="Fill Random Values", command=fill_random_values)
fill_random_button.grid(row=17, column=1, padx=10, pady=5)  # Place the Fill Random Values button to the right

# Créer un menu d'aide (onglet 2)
help_text = "Cet onglet fournit de l'aide pour utiliser l'application."
help_label = ttk.Label(tab2, text=help_text)
help_label.pack(padx=10, pady=10)

# Liaison de l'événement de changement d'onglet à la fonction tab_changed
notebook.bind("<<NotebookTabChanged>>", tab_changed)

# Lancer la boucle principale de l'interface utilisateur
root.mainloop()


E:\anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
